In [36]:
from pyiem.plot.use_agg import plt
from pyiem.plot import get_cmap
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.colors as mpcolors
import datetime
import pytz
from zoneinfo import ZoneInfo
from pandas.io.sql import read_sql
from pyiem.util import get_sqlalchemy_conn
cmap = get_cmap("gist_rainbow")
cmap.set_under("tan")
norm = mpcolors.BoundaryNorm(
    list(range(11)), 256
)


In [20]:
with get_sqlalchemy_conn("iem") as conn:
    df = read_sql("""SELECT valid, vsby, ugc_zone,
     id from current_log c JOIN stations t ON (t.iemid = c.iemid)
     WHERE t.network = 'IA_ASOS' and valid > '2022-04-21 00:00'
     and vsby is not null 
     ORDER by valid ASC
     """, conn, index_col=None)

In [23]:
with get_sqlalchemy_conn("postgis") as conn:
    warndf = read_sql("""SELECT issue, expire, ugc from warnings_2022 where issue > '2022-04-20 12:00'
    and issue < '2022-04-22' and substr(ugc, 1, 2) = 'IA' and phenomena = 'FG' and significance = 'Y'
     """, conn, index_col=None)

In [24]:
df2 = df.merge(warndf, left_on="ugc_zone", right_on="ugc", how="left")
df2["warned"] = (df2["issue"] < df2["valid"]) & (df2["expire"] > df2["valid"])
df2

,valid,vsby,ugc_zone,id,issue,expire,ugc,warned
0,2022-04-20 17:04:00+00:00,3.0,IAZ301,SPW,NaT,NaT,NaN,False
1,2022-04-20 17:04:00+00:00,4.0,IAZ035,FOD,2022-04-21 04:16:00+00:00,2022-04-21 13:51:00+00:00,IAZ035,False
2,2022-04-20 17:05:00+00:00,2.5,IAZ060,DSM,2022-04-21 04:16:00+00:00,2022-04-21 16:00:00+00:00,IAZ060,False
3,2022-04-20 17:05:00+00:00,10.0,IAZ029,OLZ,NaT,NaT,NaN,False
4,2022-04-20 17:05:00+00:00,10.0,IAZ090,SDA,2022-04-21 04:08:00+00:00,2022-04-21 16:00:00+00:00,IAZ090,False
...,...,...,...,...,...,...,...,...
36390,2022-04-22 03:15:00+00:00,10.0,IAZ040,IIB,NaT,NaT,NaN,False
36391,2022-04-22 03:15:00+00:00,10.0,IAZ074,OXV,2022-04-21 08:31:00+00:00,2022-04-21 16:00:00+00:00,IAZ074,False
36392,2022-04-22 03:15:00+00:00,10.0,IAZ018,CCY,2022-04-21 07:59:00+00:00,2022-04-21 13:00:00+00:00,IAZ018,False
36393,2022-04-22 03:15:00+00:00,10.0,IAZ037,IFA,2022-04-21 08:31:00+00:00,2022-04-21 13:51:00+00:00,IAZ037,False


In [3]:
x = []
y = []
base = datetime.datetime(2021, 11, 15, 10).replace(tzinfo=pytz.utc)
for hr in range(24):
    sts = base + datetime.timedelta(hours=hr)
    ets = sts + datetime.timedelta(hours=1)
    x.append(sts)
    df2 = df[(df['valid'] >= sts) & (df['valid'] < ets)]
    df2 = df2.sort_values('tmpf')
    y.append(df2['tmpf'].max() - df2['tmpf'].min())
    print("%s %s %s %s" % (hr, df2[['id', 'tmpf', 'valid']].head(1), df2[['id', 'tmpf', 'valid']].tail(1), y[-1]))


0       id  tmpf                     valid
742  CWI  28.0 2021-11-15 10:56:00+00:00       id  tmpf                     valid
611  SDA  44.6 2021-11-15 10:15:00+00:00 16.6
1       id  tmpf                     valid
842  MCW  28.0 2021-11-15 11:53:00+00:00       id  tmpf                     valid
818  RDK  41.0 2021-11-15 11:35:00+00:00 13.0
2       id  tmpf                     valid
983  EST  28.0 2021-11-15 12:52:00+00:00        id  tmpf                     valid
1001  RDK  39.2 2021-11-15 12:55:00+00:00 11.200000000000003
3        id  tmpf                     valid
1088  CAV  28.4 2021-11-15 13:35:00+00:00        id  tmpf                     valid
1150  SDA  39.2 2021-11-15 13:55:00+00:00 10.800000000000004
4        id  tmpf                     valid
1244  DEH  28.4 2021-11-15 14:35:00+00:00        id  tmpf                     valid
1334  SDA  46.4 2021-11-15 14:55:00+00:00 18.0
5        id  tmpf                     valid
1393  DEH  30.2 2021-11-15 15:35:00+00:00        id  tmpf      

In [54]:
sts = datetime.datetime(2022, 4, 20, 21, tzinfo=ZoneInfo("America/Chicago"))
ets = datetime.datetime(2022, 4, 21, 12, tzinfo=ZoneInfo("America/Chicago"))
interval = datetime.timedelta(hours=1)
xticks = []
xticklabels = []
now = sts
while now <= ets:
    if now.hour % 3 != 0:
        now += interval
        continue
    fmt = "%-I %p" if now.hour != 0 else '%-I %p\n%-d %b'
    xticks.append(now)
    xticklabels.append(now.strftime(fmt))
    now += interval

(fig, axes) = plt.subplots(1, 1, figsize=(12, 6.75), facecolor='white')
ax = axes
ames = df2
names = {'AMW': "Ames", "DSM": "Des Moines", "LWD": "Lamoni", "CID": "Cedar Rapids"}
for i, stid in enumerate(['AMW', 'DSM', 'LWD', 'CID']):
    color = cmap(i / 5.)
    ames = df2[df2["id"] == stid]
    ax.scatter(ames['valid'].values, ames['vsby'].values, color=color, marker='o', alpha=1,
               label=f'[{stid}] {names[stid]}', s=40)

    df3 = ames[ames["warned"]]
    ax.scatter(df3['valid'].values, df3['vsby'].values, color='k', marker='+', alpha=1)

ax.set_title("20-21 April 2022 Iowa Airport Visibility Observations", fontsize=24)
ax.set_ylabel("Horizontal Visibility [mile]", fontsize=16)
ax.legend(ncol=10, loc=2)
ax.text(1, -0.1, 'dot with "+" indicates under NWS Dense Fog Advisory',
        transform=ax.transAxes, ha='right', fontsize=12)
#ax.set_yticks([0, 45, 90, 135, 180, 225, 270, 315, 360])
#ax.set_yticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N'], fontsize=14)
#ax.set_ylim(-1, 361)
#leg = ax.legend()
#for lh in leg.legendHandles: 
#    lh.set_alpha(1)
ax.grid(True)
ax.set_ylim(0, 12)
"""
x = []
y = []
for dt, gdf in df.groupby(df['valid'].dt.strftime("%Y%m%d%H")):
    rng = gdf['tmpf'].max() - gdf['tmpf'].min()
    if rng < 10:
        continue
    x.append(datetime.datetime.strptime(dt, '%Y%m%d%H'))
    y.append(rng)

ax2 = ax.twinx()
ax2.plot(x, y, color='white', zorder=5, lw=4)
ax2.plot(x, y, color='r', zorder=5, lw=3)
ax2.set_xticks([])
ax2.set_ylim(0, 60)
ax2.tick_params(axis='y', colors='red')
ax2.set_ylabel("Air Temperature Range $^\circ$F", color='r')
"""
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, fontsize=18)
ax.set_xlim(sts, ets)


#ax = axes[1]
#ax.plot(x, y)
#ax.grid(True)
#ax.set_ylabel("Temperature Spread $^\circ$F")
fig.savefig('220422.png')
plt.close()